In [2]:
import pandas as pd
import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import os
import numpy as np
import requests
from bs4 import BeautifulSoup

In [25]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
           'Accept-Language': 'en-US, en;q=0.5'})


movie_name = []
year = []
time=[]
rating=[]
metascore =[]
director=[]
votes = []
gross = []
description = []
genre=[]
cast=[]
cas=[]
pages = np.arange(1,3000,50)
#https://www.imdb.com/search/title/?title_type=feature&primary_language=en
#https://www.imdb.com/search/title/?title_type=feature&primary_language=en&ref_=adv_prv
for page in pages:
   
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&primary_language=en&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text.replace('(', '')
        year_of_release=year_of_release.replace(')','')
        year.append(year_of_release)
        
        runtime = store.p.find("span", class_ = 'runtime').text if store.find('span', class_ = "runtime") else "NA"
        time.append(runtime)
        
        gen = store.p.find("span", class_ = 'genre').text.replace('\n', '') if store.find('span', class_ = "genre") else "NA"
        genre.append(gen)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '') if store.find('div', class_ = "inline-block ratings-imdb-rating") else "NA"
        rating.append(rate)
        #rate = store.find('div', class_ = "ratings-bar").find('strong').text.replace('\n', '')
        #rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else "NA"#if meta score not present then *
        
        metascore.append(meta)
        
        #dire=store.find('p',class_ = "metascore")
        dire=store.find('p',class_='').find_all('a')[0].text
        
        director.append(dire)
        
        #cas=([a.text for a in store.find('p',class_='').find_all('a')[1:]])
        #cast=','.join(map(str,cas))
        cast.append([a.text for a in store.find('p',class_='').find_all('a')[1:]])
      
        value = store.find_all('span', attrs = {'name':'nv'}) if store.find_all('span', attrs = {'name':'nv'}) else 'NA'
        vote = value[0].text if store.find_all('span', attrs = {'name':'nv'}) else 'NA'

        votes.append(vote)
        
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else 'NA'
        description.append(description_)
        
#dataframe
for i in cast:
    c=','.join(map(str,i))
    cas.append(c)
   
movie_list = pd.DataFrame({ "Movie Name": movie_name, "Year of Release" : year, "Watch Time": time, "Genre":genre, "Movie Rating": rating, "Metascore of movie": metascore,"Director":director,"Cast":cas,"Votes" : votes,"Description": description})
#movie_list = pd.DataFrame({ "Movie Name": movie_name, "Year of Release" : year, "Watch Time": time,"Movie Rating": rating, "Metascore of movie": metascore,"Director":director,"Cast":cas,"Votes" : votes,"Description": description})


movie_list.to_csv("movie data_new_6.csv")
movie_list.head()

,Movie Name,Year of Release,Watch Time,Genre,Movie Rating,Metascore of movie,Director,Cast,Votes,Description
0,The Batman,2022,176 min,"Action, Crime, Drama",8.1,72,Matt Reeves,"Robert Pattinson,Zoë Kravitz,Jeffrey Wright,Co...","401,811","When the Riddler, a sadistic serial killer, be..."
1,The Northman,2022,136 min,"Action, Adventure, Drama",7.9,83,Robert Eggers,"Alexander Skarsgård,Nicole Kidman,Claes Bang,E...","22,952",From visionary director Robert Eggers comes Th...
2,Thor: Love and Thunder,2022,NA,"Action, Adventure, Fantasy",NA,NA,Taika Waititi,"Taika Waititi,Karen Gillan,Natalie Portman,Chr...",NA,"Thor enlists the help of Valkyrie, Korg and ex..."
3,Fantastic Beasts: The Secrets of Dumbledore,2022,142 min,"Adventure, Family, Fantasy",6.5,47,David Yates,"Eddie Redmayne,Jude Law,Ezra Miller,Dan Fogler","39,317",Albus Dumbledore assigns Newt and his allies w...
4,Everything Everywhere All at Once,2022,139 min,"Action, Adventure, Comedy",8.9,82,Dan Kwan,"Daniel Scheinert,Michelle Yeoh,Stephanie Hsu,K...","24,150",An aging Chinese immigrant is swept up in an i...


In [24]:
movie_list
#pip install snowflake
#pip install snowflake-connector-python
#pip install snowflake-sqlalchemy

,Movie Name,Year of Release,Watch Time,Genre,Movie Rating,Metascore of movie,Director,Cast,Votes,Description
0,The Batman,2022,176 min,"Action, Crime, Drama",8.1,72,Matt Reeves,"Robert Pattinson,Zoë Kravitz,Jeffrey Wright,Co...","401,811","When the Riddler, a sadistic serial killer, be..."
1,The Northman,2022,136 min,"Action, Adventure, Drama",7.9,83,Robert Eggers,"Alexander Skarsgård,Nicole Kidman,Claes Bang,E...","22,952",From visionary director Robert Eggers comes Th...
2,Thor: Love and Thunder,2022,NA,"Action, Adventure, Fantasy",NA,NA,Taika Waititi,"Taika Waititi,Karen Gillan,Natalie Portman,Chr...",NA,"Thor enlists the help of Valkyrie, Korg and ex..."
3,Fantastic Beasts: The Secrets of Dumbledore,2022,142 min,"Adventure, Family, Fantasy",6.5,47,David Yates,"Eddie Redmayne,Jude Law,Ezra Miller,Dan Fogler","39,317",Albus Dumbledore assigns Newt and his allies w...
4,Everything Everywhere All at Once,2022,139 min,"Action, Adventure, Comedy",8.9,82,Dan Kwan,"Daniel Scheinert,Michelle Yeoh,Stephanie Hsu,K...","24,150",An aging Chinese immigrant is swept up in an i...
...,...,...,...,...,...,...,...,...,...,...
995,Greyhound,2020,91 min,"Action, Drama, History",7.0,64,Aaron Schneider,"Tom Hanks,Elisabeth Shue,Stephen Graham,Matt Helm","92,383",Several months after the U.S. entry into World...
996,Burnt,I 2015,101 min,"Comedy, Drama",6.6,42,John Wells,"Bradley Cooper,Sienna Miller,Daniel Brühl,Ricc...","115,140",Adam Jones is a chef who destroyed his career ...
997,The Purge,I 2013,85 min,"Horror, Sci-Fi, Thriller",5.7,41,James DeMonaco,"Ethan Hawke,Lena Headey,Max Burkholder,Adelaid...","219,108",A wealthy family is held hostage for harboring...
998,The 40-Year-Old Virgin,2005,116 min,"Comedy, Romance",7.1,73,Judd Apatow,"Steve Carell,Catherine Keener,Paul Rudd,Romany...","426,486","Goaded by his buddies, a nerdy guy who's never..."


In [42]:
os.environ["SF_USER"] = "GUEST"
os.environ["SF_PASSWORD"] = "Password1"
os.environ["SF_ACCOUNT"] = "wp95471.us-central1.gcp"
os.environ["SF_SCHEMA"] = "DATALAKE"
os.environ["SF_WAREHOUSE"] = "COMPUTE_WH"
os.environ["SF_DATABASE"] = "TESTDATA"
os.environ["SF_ROLE"] = "ACCOUNTADMIN"

In [43]:
engine = create_engine(URL(
            account = os.environ["SF_ACCOUNT"],
            user = os.environ["SF_USER"],
            password = os.environ["SF_PASSWORD"],
            warehouse = os.environ["SF_WAREHOUSE"],
            role = os.environ["SF_ROLE"],
            database = os.environ["SF_DATABASE"],
            schema = os.environ["SF_SCHEMA"],
        ))

In [44]:
connection = engine.connect()

In [45]:
try:
    connection = engine.connect()
    results = connection.execute('select current_version()').fetchone()
    print(results[0])
finally:
    connection.close()
    engine.dispose()

6.13.1


In [46]:

movie_list.to_sql('NEW_MOVIE_DATA', con = engine, index = False, if_exists = 'append', chunksize = 16000)



/Users/rishiupadhyay/opt/anaconda3/lib/python3.8/site-packages/snowflake/sqlalchemy/snowdialect.py:206: SAWarning: Dialect snowflake:snowflake will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  results = connection.execute(
/Users/rishiupadhyay/opt/anaconda3/lib/python3.8/site-packages/pandas/io/sql.py:1333: UserWarning: The provided table name 'NEW_MOVIE_DATA' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table n

In [50]:
data= pd.read_sql_query("""
select *
from TESTDATA.DATALAKE.NEW_MOVIE_DATA

""", engine)

/Users/rishiupadhyay/opt/anaconda3/lib/python3.8/site-packages/cryptography/hazmat/backends/openssl/x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


In [51]:
data

,Movie Name,Year of Release,Watch Time,Genre,Movie Rating,Metascore of movie,Director,Cast,Votes,Description
0,The Batman,2022,176 min,"Action, Crime, Drama",8.1,72,Matt Reeves,"Robert Pattinson,Zoë Kravitz,Jeffrey Wright,Co...","401,811","When the Riddler, a sadistic serial killer, be..."
1,The Northman,2022,136 min,"Action, Adventure, Drama",7.9,83,Robert Eggers,"Alexander Skarsgård,Nicole Kidman,Claes Bang,E...","22,952",From visionary director Robert Eggers comes Th...
2,Thor: Love and Thunder,2022,NA,"Action, Adventure, Fantasy",NA,NA,Taika Waititi,"Taika Waititi,Karen Gillan,Natalie Portman,Chr...",NA,"Thor enlists the help of Valkyrie, Korg and ex..."
3,Fantastic Beasts: The Secrets of Dumbledore,2022,142 min,"Adventure, Family, Fantasy",6.5,47,David Yates,"Eddie Redmayne,Jude Law,Ezra Miller,Dan Fogler","39,317",Albus Dumbledore assigns Newt and his allies w...
4,Everything Everywhere All at Once,2022,139 min,"Action, Adventure, Comedy",8.9,82,Dan Kwan,"Daniel Scheinert,Michelle Yeoh,Stephanie Hsu,K...","24,150",An aging Chinese immigrant is swept up in an i...
...,...,...,...,...,...,...,...,...,...,...
2995,Patrick the Pug,I 2018,94 min,"Comedy, Family, Romance",5.9,35,Mandie Fletcher,"Sam Fletcher,Beattie Edmondson,Rupert Holliday...","3,646",Sarah is a young woman whose life is in a bit ...
2996,Fright Night,1985,106 min,"Fantasy, Horror",7.0,62,Tom Holland,"Chris Sarandon,William Ragsdale,Amanda Bearse,...","68,316",A teenager discovers that the newcomer in his ...
2997,How Do You Know,2010,121 min,"Comedy, Drama, Romance",5.4,46,James L. Brooks,"Reese Witherspoon,Paul Rudd,Owen Wilson,Jack N...","49,604",After being cut from the U.S.A. softball team ...
2998,The Dark and the Wicked,2020,95 min,Horror,6.1,72,Bryan Bertino,"Marin Ireland,Michael Abbott Jr.,Julie Oliver-...","15,420",On a secluded farm in a nondescript rural town...


In [55]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle


data.rename(columns={'Unnamed: 0': 'movie_id'}, inplace=True)
data.rename(columns={'Movie Name': 'Title'}, inplace=True)
data.rename(columns={'Year of Release': 'Year'}, inplace=True)


columns=['Cast','Director','Genre','Title','Description']


data[columns].isnull().values.any()#no null values

def get_important_features(data):
    important_features=[]
    for i in range (0,data.shape[0]):
        important_features.append(data['Title'][i]+' '+data['Director'][i]+' '+data['Genre'][i]+' '+data['Description'][i]+' '+data['Cast'])
    return important_features


#creating a column to hold the combined strings
data['important_features']=get_important_features(data)


tfidf = TfidfVectorizer(stop_words='english')
#data['Description'] = data['Description'].fillna('')

tfidf_matrix = tfidf.fit_transform(data['important_features'].astype("str"))
tfidf_matrix.shape
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


indices = pd.Series(data.index, index=data['Title']).drop_duplicates()
#indices['Stillwater']
#sim_scores = list(enumerate(cosine_sim[indices['Stillwater']]))
def get_recommendations(Title, cosine_sim=cosine_sim):
    idx = indices[Title]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 5 most similar movies
    movies=data['Title'].iloc[movie_indices]
    id=data['movie_id'].iloc[movie_indices]
    dict={"Movies":movies,"id":id}
    final_df=pd.DataFrame(dict)
    final_df.reset_index(drop=True,inplace=True)
    return final_df

data.info()
new = data.drop(columns=['Year','Watch Time','Genre','Movie Rating','Metascore of movie','Director','Cast','Votes','Description'])
new
pickle.dump(new,open('movie_list_new.pkl','wb'))
pickle.dump(cosine_sim,open('similarity_new.pkl','wb'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Title               3000 non-null   object
 1   Year                3000 non-null   object
 2   Watch Time          3000 non-null   object
 3   Genre               3000 non-null   object
 4   Movie Rating        3000 non-null   object
 5   Metascore of movie  3000 non-null   object
 6   Director            3000 non-null   object
 7   Cast                3000 non-null   object
 8   Votes               3000 non-null   object
 9   Description         3000 non-null   object
 10  important_features  3000 non-null   object
dtypes: object(11)
memory usage: 257.9+ KB


In [56]:
def get_important_features(data):
    important_features=[]
    for i in range (0,data.shape[0]):
        important_features.append(data['Genre'][i])
    return important_features


#creating a column to hold the combined strings
data['important_features']=get_important_features(data)


tfidf = TfidfVectorizer(stop_words='english')
#data['Description'] = data['Description'].fillna('')

tfidf_matrix = tfidf.fit_transform(data['important_features'].astype("str"))
tfidf_matrix.shape
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


indices = pd.Series(data.index, index=data['Title']).drop_duplicates()
#indices['Stillwater']
#sim_scores = list(enumerate(cosine_sim[indices['Stillwater']]))
def get_recommendations(Title, cosine_sim=cosine_sim):
    idx = indices[Title]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 5 most similar movies
    movies=data['Title'].iloc[movie_indices]
    id=data['movie_id'].iloc[movie_indices]
    dict={"Movies":movies,"id":id}
    final_df=pd.DataFrame(dict)
    final_df.reset_index(drop=True,inplace=True)
    return final_df

data.info()
new = data.drop(columns=['Year','Watch Time','Genre','Movie Rating','Metascore of movie','Director','Cast','Votes','Description'])
new
pickle.dump(new,open('movie_listgenre_new.pkl','wb'))
pickle.dump(cosine_sim,open('similaritygenre_new.pkl','wb'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Title               3000 non-null   object
 1   Year                3000 non-null   object
 2   Watch Time          3000 non-null   object
 3   Genre               3000 non-null   object
 4   Movie Rating        3000 non-null   object
 5   Metascore of movie  3000 non-null   object
 6   Director            3000 non-null   object
 7   Cast                3000 non-null   object
 8   Votes               3000 non-null   object
 9   Description         3000 non-null   object
 10  important_features  3000 non-null   object
dtypes: object(11)
memory usage: 257.9+ KB
